In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Colab_Notebooks/SI 507/SI507_Final_project_code/')

Secrets files with API key info

In [ ]:
import secrets_t
import secrets_r

In [ ]:
import os
import math
import json
import requests
from requests_oauthlib import OAuth1
from datetime import datetime

In [ ]:
client_key = secrets_t.TWITTER_API_KEY
client_secret = secrets_t.TWITTER_API_KEY_SECRET
access_token = secrets_t.TWITTER_ACCESS_TOKEN
access_token_secret = secrets_t.TWITTER_ACCESS_TOKEN_SECRET

In [ ]:
oauth = OAuth1(client_key,
            client_secret=client_secret,
            resource_owner_key=access_token,
            resource_owner_secret=access_token_secret)

Just for Testing!

In [ ]:
endpoint_url = 'https://api.twitter.com/1.1/search/tweets.json'

params = {'q': 'worldcupqatar'}
response_twitter = requests.get(endpoint_url, 
                        params=params, 
                        auth=oauth)
print (response_twitter.status_code)
# print(response.json()['statuses'])

200


Reddit API

In [ ]:
client_id = secrets_r.personal_use_script
secret_token = secrets_r.secret_token
reddit_pwd = secrets_r.password

In [ ]:
auth = requests.auth.HTTPBasicAuth(client_id, secret_token)

In [ ]:
data = {'grant_type': 'password',
        'username': 'ClubIntelligent5370',
        'password': reddit_pwd}

In [ ]:
headers = {'User-Agent': 'chatterbot507/0.0.1'}

# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

# add authorization to our headers dictionary
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}

# while the token is valid (~2 hours) we just add headers=headers to our requests
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

<Response [200]>

In [ ]:
params = {'q': 'worldcupqatar'}
response_reddit = requests.get("https://oauth.reddit.com/search?", params=params,
                   headers=headers)

# print(res.json()['data']['children'])  # let's see what we get

In [ ]:
class Tweet:
  def __init__(self, record):
    self.created_at = record['created_at']
    self.id = record['id_str']
    self.text = record['text']
    try:
      self.url = record['entities']['urls'][0]['url']
    except IndexError:
      self.url = None
    self.user = dict()
    self.user['user_id'] = record['user']['id_str']
    self.user['name'] = record['user']['name']
    self.coordinates = record['coordinates']
    # self.reply_count = record['reply_count']
    self.retweet_count = record['retweet_count']
    self.favorite_count = record['favorite_count']
    self.hashtags = record['entities']['hashtags']


In [ ]:
class Post:
  def __init__(self, record):
    self.id = record['data']['id']
    self.created_utc = record['data']['created_utc']
    self.title = record['data']['title']
    self.url = record['data']['url']
    self.selftext = record['data']['selftext']
    self.ups = record['data']['ups']
    self.downs = record['data']['downs']
    self.upvote_ratio = record['data']['upvote_ratio']
    self.num_comments = record['data']['num_comments']
    self.score = record['data']['score']


In [ ]:
def create_cache(object_list, json_cache_path):
  """Caches all attributes of the Tweet and Post objects in json format

  Args:
      object_list (List[Tweet/Post]): 
        List of Tweet/Post objects to cache
      cache_file_path (str): 
        Path to json cache file
  """
  json_data = {}
  json_data['cache_creation_time'] = str(datetime.now())
  json_data['data'] = dict()
  for object in object_list:
    json_data['data'][object.id] = object.__dict__
  with open(json_cache_path, "w") as cfile:
    json.dump(json_data, cfile)

In [ ]:
def load_from_cache(json_cache_path):
  """Loads the objects of Tweet/Post into a json object.

  Args:
      cache_file_path (str): 
        Path to json cache file

  Returns:
      _type_: Dict{Dict{...}}
  """
  if os.path.exists(json_cache_path) and os.path.getsize(json_cache_path) != 0:
    with open(json_cache_path, "r") as cfile:
      json_data = json.load(cfile)
  else:
    print("Please provide a valid json cache")
    return None
  return json_data

In [ ]:
Tweet_list = [Tweet(record) for record in response_twitter.json()['statuses']]

In [ ]:
Post_list = [Post(record) for record in response_reddit.json()['data']['children']]

In [ ]:
TWITTER_JSON = 'twitter_tweets.json'
REDDIT_JSON = 'reddit_posts.json'
create_cache(Tweet_list, TWITTER_JSON)
create_cache(Post_list, REDDIT_JSON)

In [ ]:
json_twitter = load_from_cache(TWITTER_JSON)
json_reddit = load_from_cache(REDDIT_JSON)

In [ ]:
# def cache_age(cache_time):
#   """Return time in hours that has passed since cache creation

#   Args: 
#       cache_time (str):
#         Time in utc of cache creation
  
#   Returns:
#       _type_: int 
#   """
#   time_diff = datetime.now() - datetime.strptime(cache_time)
#   return time_diff.total_seconds() / 3600

# def create_tweets(json_cache_path):
#   cached_json = load_from_cache(json_cache_path)
#   if cached_json is None or cache_age(cached_json['cache_creation_time']) > 1:
#     params = {'q': 'worldcupqatar'}
#     response_twitter = requests.get('https://api.twitter.com/1.1/search/tweets.json', 
#                             params=params, 
#                             auth=oauth)
#     response_reddit = requests.get("https://oauth.reddit.com/search?", params=params,
#                       headers=headers)
#     Tweet_list = [Tweet(record) for record in response_twitter.json()['statuses']]
#     Post_list = [Post(record) for record in response_reddit.json()['data']['children']]

#     with open(json_cache_path, "r") as cfile:
#       json_data = json.load(cfile)
    
#     for district in Districts:
#       district.RandomLat = json_data[district.name]["RandomLat"]    
#       district.RandomLong = json_data[district.name]["RandomLong"] 


Tree Structure

In [ ]:
def hotness(time, score):
  """ Takes time in utc and score as arguments
      (i.e. (upvotes - downvotes) for reddit and favorite_count for twitter)
      and calculates the hotness value of a post
  Args:
      time: int
      score: int
  Returns:
      __type__: int
  """
  if score <= 0:
    return time
  else:
    return time + math.log10(score)

class TreeNode:
  def __init__(self, time, comments, reposts, retweets, score, object):
    self.nodeVal = tuple(time, comments, reposts, hotness(time, score))
    self.object_ref = object
    self.left = None
    self.right = None

class kdTree:
  def __init__(self):
    self.root = None

  # inserts a new node into tree if node does not exist
  def insertNode(self, newNode):
    pass

  # finds maximum val in dth dimension
  def findMaxD(self, d, depth):
    pass
  
  def findMinD(self, d, depth):
    pass

  def deleteNode(self, nodeId):
    pass

Flask Code

In [ ]:
# from flask import Flask
# app = Flask(__name__)

# @app.route('/')
# def index():
#   return f"<h1> Hello World! </h1> "

In [ ]:
# app.run(debug=True)